## 0. Librerías necesarias

In [14]:
# 1. Para análisis de datos:
import pandas as pd
import numpy as np
# 2. Para visualizaciones
import matplotlib.pyplot as plt
import seaborn as sns
# 3. Para procesamiento lingüístico con spaCy
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.tokens import Doc, Span, Token
# 4. Para análisis de sentimientos y otras extensiones
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
nltk.download('vader_lexicon')
nltk.download('stopwords')
# 5. Topic modeling (gensim)
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
# 6. BERTopic
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
# 7. Utilidades
import re
import glob
import os


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/clarabueno/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/clarabueno/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 1. Recolección del texto

### 1.1 Limpieza de .txt de Proyecto Gutenberg

In [15]:
# 1. Limpieza básica de Gutenberg, reaprovechable para otros textos

def limpiar_gutenberg(texto):
    # Eliminar todo lo que aparece antes del marcador *** START OF
    patron_inicio = r"\*\*\* START OF.*?\*\*\*"
    patron_fin = r"\*\*\* END OF.*?\*\*\*"
    
    # Buscar el inicio del contenido real
    inicio = re.search(patron_inicio, texto, flags=re.IGNORECASE | re.DOTALL)
    if inicio:
        texto = texto[inicio.end():]  # Cortar hasta el final del marcador START
    
    # Buscar el final del contenido real
    fin = re.search(patron_fin, texto, flags=re.IGNORECASE | re.DOTALL)
    if fin:
        texto = texto[:fin.start()]  # Cortar antes del marcador END
    
    # Reducir saltos de línea excesivos (triple salto → doble salto)
    texto = re.sub(r"\n\s*\n\s*\n+", "\n\n", texto)
    
    return texto.strip()

# 2. Limpieza + eliminar título/autora, reaprovechable con modificaciones

def limpiar_gutenberg_y_metadatos(texto):
    # Primero aplicamos la limpieza estándar
    texto = limpiar_gutenberg(texto)

    # Patrones que identifican líneas de encabezado o metadatos
    patrones_eliminar = [
        r"\blolly willowes\b",
        r"\bby sylvia townsend warner\b",
        r"\bsylvia townsend warner\b"
    ]

    lineas_limpias = []
    for linea in texto.splitlines():
        linea_sin_espacios = linea.strip()

        # 1. Las líneas de encabezado suelen ser muy breves
        linea_corta = len(linea_sin_espacios.split()) <= 6

        # 2. Coinciden con alguno de los patrones de metadatos
        es_metadato = any(
            re.search(patron, linea_sin_espacios, flags=re.IGNORECASE)
            for patron in patrones_eliminar
        )

        if linea_corta and es_metadato:
            continue  # Eliminamos esa línea de encabezado
        
        lineas_limpias.append(linea)

    texto = "\n".join(lineas_limpias)

    # Normalizar saltos de línea
    texto = re.sub(r"\n\s*\n\s*\n+", "\n\n", texto)
    return texto.strip()

# 3. Limpieza completa (reparación de guiones, OCR, metadatos)

def limpiar_gutenberg_completo(texto):
    # 1. Limpiar Gutenberg + encabezados/título/autora
    texto = limpiar_gutenberg_y_metadatos(texto)

    # 2. Reparar palabras cortadas con guión al final de la línea:
    #    "con-\ntinente" → "continente"
    texto = re.sub(r"-\s*\n\s*", "", texto)

    # 3. Reparar saltos de línea dentro de frases
    #    (unir líneas cuando la anterior no termina en . ! ?)
    texto = re.sub(r"(?<![.!?])\n(?!\n)", " ", texto)

    # 4. Normalizar saltos entre párrafos (dejar doble salto limpio)
    texto = re.sub(r"\n\s*\n", "\n\n", texto)

    # 5. Eliminar espacios múltiples consecutivos
    texto = re.sub(r" {2,}", " ", texto)

    # 6. Eliminar espacios del principio y final
    return texto.strip()


### 1.2 Cargamos el texto

In [16]:
# Nos traemos el texto que queremos analizar:
with open("../data/lolly_willowes.txt", "r", encoding="utf-8") as l:
    raw_lolly = l.read()

lolly = limpiar_gutenberg_completo(raw_lolly)

# Lo guardamos:
with open("../data/lolly_clean.txt", "w", encoding="utf-8") as f:
    f.write(lolly)

In [17]:
# Ahora "lolly" es una cadena larguísima de texto, así que necesitamos hacer las pertinentes modificaciones
display(type(lolly))
display(len(lolly))

str

289977

## 2. SpaCy para limpiar, tokenizar y lematizar

In [30]:
# Cargamos el modelo
nlp = spacy.load("en_core_web_sm")

# Stopwords personalizadas (partimos de las estándar)
custom_stopwords = set(STOP_WORDS)

# Palabras que queremos conservar aunque sean stopwords habituales
wtk = {
    "not", "no", "never",
    "yet", "still", "though",
    "perhaps", "maybe"
}

# Las retiramos de las stopwords personalizadas
for w in wtk:
    custom_stopwords.discard(w)

# Función auxiliar (si quieres seguir usándola)
def clean_tokens(lolly):
    doc = nlp(lolly)
    return [
        token.lemma_.lower()
        for token in doc
        if token.is_alpha and token.lemma_.lower() not in custom_stopwords
    ]


# Función que devuelve el DF completo

def procesar_texto_df(texto):
    """
    Procesa un texto con spaCy y devuelve un DataFrame con información lingüística.
    Aplica las stopwords personalizadas definidas en custom_stopwords.
    
    Parámetros
    ----------
    texto : str
        Texto ya limpio.

    Retorna
    -------
    pandas.DataFrame
        Un DataFrame con columnas:
        - token: forma original
        - lema: forma lematizada en minúsculas
        - pos: categoría morfosintáctica
        - tag: etiqueta morfológica detallada
        - dep: tipo de dependencia sintáctica
        - es_stop_custom: si está en tus stopwords personalizadas
        - es_stop_spacy: si spaCy la marca como stopword
    """
    doc = nlp(texto)

    rows = []
    for token in doc:
        if not token.is_alpha:
            continue

        lema = token.lemma_.lower()
        rows.append({
            "token": token.text,
            "lema": lema,
            "pos": token.pos_,
            "tag": token.tag_,
            "dep": token.dep_,
            "es_stop_custom": lema in custom_stopwords,
            "es_stop_spacy": token.is_stop
        })

    return pd.DataFrame(rows)


## 3. Transformación en DF y cotilleo superficial

In [31]:
tokens = clean_tokens(lolly)
df_tokens = pd.DataFrame(tokens, columns=["token"])

In [32]:
# Contar los tokens totales 
display(df_tokens.shape)
# Ver ejemplos aleatorios
display(df_tokens.sample(20))

(22152, 1)

,token
9446,spot
6126,leave
14189,path
11327,screen
17054,demetrius
20881,believe
1738,home
4582,morning
22017,catch
12126,slide


In [33]:
# Describir el tamaño de las palabras para ver por qué tiene preferencia la autora
df_tokens["longitud"] = df_tokens["token"].str.len()
df_tokens["longitud"].describe()

count    22152.000000
mean         5.610599
std          2.081681
min          1.000000
25%          4.000000
50%          5.000000
75%          7.000000
max         18.000000
Name: longitud, dtype: float64

## Comparativa de la longitud media de las palabras

A continuación se presenta una referencia de longitudes medias de palabra típicas en distintos tipos de texto en inglés:

| Tipo de texto               | Longitud media |
|-----------------------------|----------------|
| Inglés conversacional       | 3.5–4.5        |
| Prensa / no ficción         | 4.5–5.0        |
| Literatura del s. XX        | 5.0–6.0        |
| Literatura del s. XIX       | 5.5–6.5        |

**Valor obtenido para Sylvia Townsend Warner:** 5.61  

---

### Percentiles del texto analizado

- **25% = 4 caracteres** → un cuarto de las palabras son muy cortas (and, with, when, will…).  
- **50% = 5 caracteres** → la palabra “típica” tiene 5 letras.  
- **75% = 7 caracteres** → solo un cuarto del vocabulario supera las 7 letras, lo cual es totalmente normal en literatura inglesa.

---

### Comparación con otras autoras

| Autora              | Media aproximada |
|---------------------|------------------|
| Jane Austen         | 5.4–5.6          |
| Emily Brontë        | 5.7–5.9          |
| Thomas Hardy        | 5.8–6.2          |
| Virginia Woolf      | 5.8–6.1          |


## Distribución léxica y hapax

In [34]:
# Distribución léxica con frecuencias básicas
frecuencias = df_tokens["token"].value_counts()
frecuencias.head(20)

token
laura       396
not         333
come        199
like        199
no          178
titus       137
caroline    127
think       127
henry       124
look        118
know        117
great       113
feel        111
day          97
time         93
little       92
old          90
place        85
willowes     83
walk         82
Name: count, dtype: int64

In [35]:
# Distribución léxica con frecuencias normalizadas
frecuencias_rel = df_tokens["token"].value_counts(normalize=True) * 100
frecuencias_rel.head(20)

token
laura       1.787649
not         1.503250
come        0.898339
like        0.898339
no          0.803539
titus       0.618454
caroline    0.573312
think       0.573312
henry       0.559769
look        0.532683
know        0.528169
great       0.510112
feel        0.501083
day         0.437884
time        0.419827
little      0.415312
old         0.406284
place       0.383713
willowes    0.374684
walk        0.370170
Name: proportion, dtype: float64

In [24]:
# ¿Encontraremos algún hapax?
hapax = frecuencias[frecuencias == 1]
display(hapax.sample(20))
display(len(hapax))


token
distinguished    1
grapple          1
distribute       1
mount            1
baking           1
inland           1
kettle           1
cope             1
thinly           1
hallowed         1
complainingly    1
hardness         1
tracery          1
celery           1
meandering       1
martial          1
townsend         1
retort           1
linoleum         1
hemlock          1
Name: count, dtype: int64

2777

## POS, TAG, DEP

In [41]:
# Procesar el texto para sacar token, lema, pos, tag, dep y si son STOP WORDS (customizado y original)
df = procesar_texto_df(lolly)


          token      lema    pos   tag        dep  es_stop_custom  \
24563       she       she   PRON   PRP      nsubj            True   
24690  striking    strike   VERB   VBG      xcomp           False   
32382       the       the    DET    DT        det            True   
25833        he        he   PRON   PRP      nsubj            True   
42936       his       his   PRON  PRP$       poss            True   
18257        It        it   PRON   PRP      nsubj            True   
44650        it        it   PRON   PRP      nsubj            True   
35759       and       and  CCONJ    CC         cc            True   
8301   sunlight  sunlight   NOUN    NN       pobj           False   
6247     greens     green   NOUN   NNS       pobj           False   
7670       that      that  SCONJ    IN       mark            True   
44371     flies       fly   NOUN   NNS  nsubjpass           False   
48646       you       you   PRON   PRP       dobj            True   
46973       his       his   PRON  

In [45]:
# Ver resultado general
display(df.sample(20))

,token,lema,pos,tag,dep,es_stop_custom,es_stop_spacy
26248,She,she,PRON,PRP,nsubj,True,True
13764,housekeeping,housekeeping,NOUN,NN,pobj,False,False
218,she,she,PRON,PRP,nsubj,True,True
44196,Lost,lost,PROPN,NNP,npadvmod,False,False
25432,well,well,ADV,RB,advmod,True,True
4474,and,and,CCONJ,CC,cc,True,True
32698,comfortable,comfortable,ADJ,JJ,amod,False,False
46929,very,very,ADV,RB,advmod,True,True
16574,this,this,DET,DT,det,True,True
3553,for,for,ADP,IN,prep,True,True


In [46]:
# Ver la frecuencia de utilización de las diferentes categorías gramaticales

df["pos"].value_counts()

pos
NOUN     8975
VERB     7244
PRON     6716
ADP      6014
DET      4885
AUX      3630
ADJ      3527
ADV      3125
PROPN    2621
CCONJ    2321
SCONJ    1493
PART     1201
NUM       239
INTJ       63
X           4
Name: count, dtype: int64

In [47]:
# Ver la frecuencia normalizada de utilización de las diferentes categorías gramaticales

df["pos"].value_counts(normalize=True) * 100

pos
NOUN     17.240386
VERB     13.915248
PRON     12.900995
ADP      11.552499
DET       9.383764
AUX       6.972992
ADJ       6.775135
ADV       6.002920
PROPN     5.034769
CCONJ     4.458489
SCONJ     2.867955
PART      2.307042
NUM       0.459103
INTJ      0.121019
X         0.007684
Name: proportion, dtype: float64